![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/annotation/english/spark-nlp-basics/playground-dataFrames.ipynb)

## 0. Colab Setup

In [ ]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 57kB/s 
     |████████████████████████████████| 204kB 31.7MB/s 
     |████████████████████████████████| 122kB 9.5MB/s 


In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline

In [ ]:
spark = sparknlp.start()

In [ ]:
document = DocumentAssembler().setInputCol('text').setOutputCol('document')

In [ ]:
tokenizer = Tokenizer().setInputCols('document').setOutputCol('token')

In [ ]:
pos = PerceptronModel.pretrained().setInputCols('document', 'token').setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[OK!]


In [ ]:
pipeline = Pipeline().setStages([document, tokenizer, pos])

In [ ]:
data = spark.read.text('./sample-sentences-en.txt').toDF('text')

AnalysisException: ignored

In [ ]:
data.show(5)

In [ ]:
model = pipeline.fit(data)

In [ ]:
result = model.transform(data)

In [ ]:
result.show(5)

In [ ]:
stored = result\
  .select('text', 'pos.begin', 'pos.end', 'pos.result', 'pos.metadata')\
  .toDF('text', 'pos_begin', 'pos_end', 'pos_result', 'pos_meta')\
  .cache()

In [ ]:
stored.printSchema()

In [ ]:
stored.show(5)

---------
## Spark SQL Functions

In [ ]:
from pyspark.sql.functions import *

In [ ]:
stored.filter(array_contains('pos_result', 'VBD')).show(5)

In [ ]:
stored.withColumn('token_count', size('pos_result')).select('pos_result', 'token_count').show(5)

In [ ]:
stored.select('text', array_max('pos_end')).show(5)

In [ ]:
stored.withColumn('unique_pos', array_distinct('pos_result')).select('pos_result', 'unique_pos').show(5)

In [ ]:
stored.groupBy(array_sort(array_distinct('pos_result'))).count().show(10)

----------------
### SQL Functions with `col`

In [ ]:
from pyspark.sql.functions import col

In [ ]:
stored.select(col('pos_meta').getItem(0).getItem('word')).show(5)

-------------
### Spark NLP Annotation UDFs

In [ ]:
result.select('pos').show(1, truncate=False)

In [ ]:
def nn_tokens(annotations):
    nn_annotations = list(
        filter(lambda annotation: annotation.result == 'NN', annotations)
    )
    return list(
        map(lambda nn_annotation: nn_annotation.metadata['word'], nn_annotations)
    )

In [ ]:
from sparknlp.functions import *

In [ ]:
from pyspark.sql.types import ArrayType, StringType

In [ ]:
result.select(map_annotations(nn_tokens, ArrayType(StringType()))('pos').alias('nn_tokens')).show(truncate=False)